scan wa page
--> request page's html
--> save the necessary part of the page into a txt file
--> scan through job links on that page
--> get the nth link -> save to a list -> request -> save file -> process -> add to df
--> k00th link -> save to csv



Cạnh tranh gần nhất:

https://pantio.vn/

https://ivymoda.com/

https://elise.vn/

(1 - 3 năm)


https://sixdo.vn/

(lâu dài)


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os
import requests
import time
from datetime import datetime
from IPython.display import clear_output
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.parse import urlparse, unquote
# os.environ['PATH'] += r"E:\Installers\chromedriver_win32\chromedriver.exe"

In [5]:
main_site_url = 'https://elise.vn/thoi-trang-nu'
urlparse(main_site_url)


ParseResult(scheme='https', netloc='elise.vn', path='/thoi-trang-nu', params='', query='', fragment='')

## Get numbered pages' url

In [2]:
path = os.getcwd()
path

'e:\\COLLEGE\\PROJECT\\Thay Nhan\\INTERN\\Crawling\\crawling_code'

In [4]:
def all_pages(main_site_url, start_page, end_page, url_format):
    all_pages_list = []
    all_pages_list += [main_site_url]
    
    for i in range(start_page, end_page+1):
        url = url_format.format(i)
        all_pages_list.append(url)
    
    parsed_url = urlparse(main_site_url)
    domain = parsed_url.netloc
    
    file_path = os.path.join(os.path.dirname(path), 'crawling_materials')
    # os.path.dirname(path) là để lùi lại thư mục Crawling
    pages_links_file_name = os.path.join(file_path, f'numbered_pages_{domain}.txt')
    
    
    with open(pages_links_file_name, 'w', encoding = 'utf-8') as f:
        for page in all_pages_list:
            f.write(f'{page}\n')
    
    return pages_links_file_name # This file stores all the links to the pagination pages

## Get all products' links

In [5]:
import re
def get_links(pagination_pages):
    wanted_links = []
    with open(pagination_pages, 'r') as f:
#         lines = f.readlines()[:2]  # Read only the first two lines to test
#         for line in lines:
        for line in f:
            page_url = line.strip()
            response = requests.get(page_url)
            html_content = response.content.decode()
            soup = BeautifulSoup(html_content, "html5lib")

            # FOR LATINA.VN
            if 'latina' in page_url:
                product_names = soup.find_all('div', class_='product-name')

                for product in product_names:
                    link = product.find('a', class_='link')['href']
                    hyperlink = "https://latina.vn" + link
                    wanted_links += [hyperlink]
            elif 'elise' in page_url:
                product
                
            print(f'Finished: {line}.')
            clear_output(wait=True)
    
    match = re.search(r'numbered_pages_(.*?)\.txt', pagination_pages)
    if match:
        domain = match.group(1)
        file_name = f'wanted_links_{domain}.txt'
        
    file_path = os.path.join(os.path.dirname(path), 'crawling_materials')
    file_wanted_links = os.path.join(file_path, file_name)
    
    with open(file_wanted_links, 'w', encoding='utf-8') as f:
        for link in wanted_links:
            f.write(f'{link}\n')

    return file_wanted_links

In [10]:
url = 'https://latina.vn/collections/all?q=filter=(!(collectionid:product=0))&page=19&view=grid'
parsed_url = urlparse(url)

# Extract the domain_folder from the netloc
domain_folder = parsed_url.netloc.split('.')[0] if parsed_url.netloc.count('.') > 0 else 'other'
domain_folder

'latina'

In [14]:
url = 'https://latina.vn/products/set-bo-thiet-ke-cao-cap-ms4236'
parsed_url = urlparse(url)

if parsed_url.netloc == 'latina.vn':
        path_parts = parsed_url.path.split('/')
        product_name = path_parts[-1]
product_name

'set-bo-thiet-ke-cao-cap-ms4236'

In [6]:
import os
from urllib.parse import urlparse, unquote
from datetime import datetime

def get_next_subfolder(day_folder):
    file_size = 1000
    subfolder_id = file_size  
    while True:
        subfolder_path = os.path.join(day_folder, str(subfolder_id))
        if not os.path.exists(subfolder_path) or len(os.listdir(subfolder_path)) < file_size:
            return subfolder_path
        subfolder_id += file_size

def file_naming(url, output_folder):
    parsed_url = urlparse(url)

    # Extract the domain_folder from the netloc
    # FOR LATINA
    domain_folder = parsed_url.netloc.split('.')[0] if parsed_url.netloc.count('.') > 0 else 'other'

    # Get the current date and time
    current_datetime = datetime.now()
    
    # Create subfolders based on year, month, and day
    page_folder = os.path.join(output_folder, domain_folder)
    year_folder = os.path.join(page_folder, str(current_datetime.year))
    month_folder = os.path.join(year_folder, str(current_datetime.month))
    day_folder = os.path.join(month_folder, str(current_datetime.day))
    
    
    subfolder_path = get_next_subfolder(day_folder)
    os.makedirs(subfolder_path, exist_ok=True)  

    if parsed_url.netloc == 'latina.vn':
        if parsed_url.netloc == 'latina.vn':
            path_parts = parsed_url.path.split('/')
            product_name = path_parts[-1]

    else:
        product_name = None

    i = 1
    while True:
        file_id = f"{i:05d}"
        file_name = f"[{current_datetime.strftime('%Y%m%d %H%M%S')}]_{file_id}_{product_name}.txt"
        file_path = os.path.join(subfolder_path, file_name)
        if not os.path.exists(file_path):
            return file_path
        i += 1

In [31]:
import requests
from bs4 import BeautifulSoup
import re
import os
from urllib.parse import urlparse
from datetime import datetime

def saving_numbered_pages(url, output_folder=r'E:\Test'):
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    
    # FOR LATINA
    if '&' not in url:
        pagenum = 'page1'
    else:
        # FOR LANINA
        pagenum = url.split('&')[-2]
        pagenum = re.sub(r'(?u)[^-\w.]', '', pagenum)
    
    parsed_url = urlparse(url)

    # Extract the domain_folder from the netloc
    # FOR LATINA
    domain_folder = parsed_url.netloc.split('.')[0] if parsed_url.netloc.count('.') > 0 else 'other'


    # Get the current date and time
    current_datetime = datetime.now()
    
    # Create subfolders based on year, month, and day
    page_folder = os.path.join(output_folder, domain_folder)
    year_folder = os.path.join(page_folder, str(current_datetime.year))
    month_folder = os.path.join(year_folder, str(current_datetime.month))
    day_folder = os.path.join(month_folder, str(current_datetime.day))
    numbered_pages_folder = os.path.join(day_folder, 'numbered pages')
    os.makedirs(numbered_pages_folder, exist_ok=True)  
    
    i = 1
    while True:
        file_id = f"{i:05d}"
        file_name = f"[{current_datetime.strftime('%Y%m%d %H%M%S')}]_{domain_folder}_{file_id}_{pagenum}.txt"
        file_path = os.path.join(numbered_pages_folder, file_name)
        if not os.path.exists(file_path):
            break
        i += 1
    
    # FOR LATINA
    if 'latina' in url:
        needed_div = soup.find('div', class_='category-products products')
        if needed_div:
            div_content = needed_div.prettify()
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(div_content)
    
    # FOR CAREER LINK
    elif 'careerlink' in url:
        needed_div = soup.find('ul', class_='list-group')
        if needed_div:
            div_content = needed_div.prettify()
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(div_content)
    
    # FOR VNWORKS
    elif 'vietnamworks' in url:
        needed_div = soup.find('div', class_='col-lg-9')
        if needed_div:
            div_content = needed_div.prettify()
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(div_content)
    
    # FOR HOTEL
    elif 'hotel' in url:
        needed_div = soup.find('div', class_='job-contain')
        if needed_div:
            div_content = needed_div.prettify()
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(div_content)
            
    return file_path

In [32]:
# with open(latina_pages, 'r') as f:
#     lines = f.readlines()
#     overall_sum = len(lines)
#     overall_count = 1
#     for line in lines:
#         page_url = line.strip()
#         print(page_url)
#         numbered_page_path = saving_numbered_pages(page_url, 'E:\COLLEGE\PROJECT\Thay Nhan\INTERN\Crawling\data')
        
#         i = 0
#         overall_progress = round((overall_count / overall_sum)*100, 2)
#         print(f'Overall progress: {overall_progress}%')
#         clear_output(wait=True)

https://latina.vn/collections/all?q=filter=(!(collectionid:product=0))&page=20&view=grid
Overall progress: 5.0%


In [16]:
def saving_detail_file(url, output_folder):
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    
    # FOR LATINA
    if 'latina' in url:
        needed_div = soup.find('section', class_='product detail-main')
    # FOR CAREER LINK
    if 'careerlink' in url:
        needed_div = soup.find('div', class_='card border-0 font-nunitosans px-4')
        
    # FOR VNWORKS
    if 'vietnamworks' in url:
        needed_div = soup.find('div', class_='page-foreground')
        
    # FOR HOTEL JOB
    if 'hotel' in url:
        needed_div = soup.find('section', class_='py-lg-4')
        
    if needed_div:
        div_content = needed_div.prettify()
        file_path = file_naming(url, output_folder)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(div_content)
        return file_path

In [17]:
import os
from urllib.parse import urlparse, unquote
from datetime import datetime


def save_csv_file(df, batch_number, domain, output_folder=r"E:\test"):
    
    current_datetime = datetime.now()
    
    # Create subfolders based on year, month, and day
    year_folder = os.path.join(output_folder, str(current_datetime.year))
    month_folder = os.path.join(year_folder, str(current_datetime.month))
    day_folder = os.path.join(month_folder, str(current_datetime.day))
    csv_folder = os.path.join(day_folder, domain)
    
    
    os.makedirs(csv_folder, exist_ok=True)  

    
    i = 1
    while True:
        file_id = f"{i:05d}"
        file_name = f"[{current_datetime.strftime('%Y%m%d %H%M%S')}]_{file_id}_batch{batch_number}.csv"
        file_path = os.path.join(csv_folder, file_name)
        if not os.path.exists(file_path):
            df.to_csv(file_path, index=False)
            return
        i += 1

In [19]:
def extract_and_save_ids(url, output_file='ids.txt'):
    id = None

    if 'latina' in url:
        id = 'latin_' + url.split('-')[-1]
    elif 'careerlink' in url:
        id = 'careerlink_' + url.split('?')[0].split('/')[-1]
    elif 'vietnamworks' in url:
        id = 'vnworks_' + url.split('&')[-2].split('=')[-1]
    elif 'hotel' in url:
        id = 'hoteljob_' + url.split('/')[-1].split('-')[0]

    if id is not None:
        try:
            with open(output_file, 'r') as file:
                existing_ids = [line.strip() for line in file]
                if id in existing_ids:
                    return False
                else:
                    with open(output_file, 'a') as file:
                        file.write(id + '\n')
                        return True
        except FileNotFoundError:
            pass



In [ ]:
with open(r'VNWork_WantedLinks_Backup.txt', 'r', encoding='utf-8') as f:
    urls = f.readlines()
for url in urls:
    extract_and_save_ids(url)

# Career Link

In [47]:
def get_summary(soup):
    sum_cols = soup.find_all('div', class_='col-6')


    test_dict = {}
    for col in sum_cols:
        sum_boxes = col.find_all('div', class_='d-flex')
        for box in sum_boxes:
            key = box.find('div', class_='summary-label')
            if key != None:
                key = key.text.strip()
                val = box.find('div', class_='font-weight-bolder')
                val = val.text.strip()
                test_dict[key] = val

        other_sum_boxes = col.find_all('div', class_='row')
        for box in other_sum_boxes:
            key = box.find('div', class_='summary-label')
            if key != None:
                key = key.text.strip()
                val_list = box.find_all('span')
                val_list = [span.text.strip() for span in val_list]
                test_dict[key] = val_list

    return test_dict

def get_info(soup):
    job_info_elements = soup.find('div', id='section-job-contact-information')
    info_boxes = job_info_elements.find_all('li', class_='d-flex')

    contact_dict = {'Contact': None, 'Location': None, 'Notes': None}
    for box in info_boxes:
        key_contact = box.find('i', class_='cli-contact-with')
        key_loc = box.find('i', class_='cli-location')
        key_notes = box.find('i', class_='cli-note')

        if key_contact != None:
            val_list = []
            label = box.find('span', class_='label').text.strip()
            person_name = box.find('span', class_='person-name').text.strip()
            val_list.append(label)
            val_list.append(person_name)
            contact_dict['Contact'] = val_list

        if key_loc != None:
            contact_loc = box.find('span', class_='align-seft-center')
            contact_loc_list = contact_loc.find_all('span')
            contact_loc_list = [span.text.strip() for span in contact_loc_list]
            contact_dict['Location'] = contact_loc_list

        if key_notes != None:
            contact_note_list = box.find_all('em')
            for i in range(len(contact_note_list)):
                contact_note_list[i] = contact_note_list[i].text.strip()
            contact_dict['Notes'] = contact_note_list

    return contact_dict

def get_keywords(soup):
    job_keyword_elements = soup.find('div', class_='tags-container')
    all_tag_chips = job_keyword_elements.find_all('a', class_='chip')
    keywords = []

    for tag_chip in all_tag_chips:
        chip_list = [tag_chip.text.strip(), "https://www.careerlink.vn/"+tag_chip.get("href")]
        keywords.append(chip_list)
        
    return keywords

def get_overview(soup):
    job_overview = soup.find('div', class_='job-overview')
    overview_boxes = job_overview.find_all('div', class_='d-flex')

    location_list = []  # Initialize the location list

    for box in overview_boxes:
        location = box.find('i', class_='cli-map-pin-line')
        salary = box.find('i', class_='cli-currency-circle-dollar')
        experience = box.find('i', class_='cli-suitcase-simple')
        post_date = box.find('i', class_='cli-calendar')

        if location != None:
            loc_spans = box.find_all('span', class_='mr-1')
            location_list = [span.text.strip() for span in loc_spans]
            city = box.find('a').text.strip()
            location_list += [city]

        if salary != None:
            job_salary = box.find('span').text.strip()

        if experience != None:
            job_exp = box.find('span').text.strip()

        if post_date != None:
            date_span = box.find('span', class_='d-none d-md-block mr-1')
            job_post_date = date_span.next_sibling.strip() if date_span else None

    return location_list, job_salary, job_exp, job_post_date



####################################
def save_to_df(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html5lib")
    
    job_title = soup.find('h1', class_='job-title').text.strip()
    
    location, job_salary, job_exp, job_post_date = get_overview(soup)
    
    org_info = soup.find('p', class_='org-name')
    a_tag = org_info.find('a')
    org_name = a_tag.span.text.strip()
    org_link = "https://www.careerlink.vn" + a_tag.get("href")
    
    job_desc_elements = soup.find('div', id='section-job-description').find_all('p')
    job_desc = [p.text.strip() for p in job_desc_elements]
    
    job_benefits_elements = soup.find('div', id='section-job-benefits')
    if job_benefits_elements:
        job_benefits_elements = job_benefits_elements.find_all('span')
        job_benefits = [span.text.strip() for span in job_benefits_elements]
    else:
        job_benefits = None

    
    job_skills_elements = soup.find('div', id='section-job-skills').find_all('p')
    job_skills = [p.text.strip() for p in job_skills_elements]
    
    job_summary = get_summary(soup)
    contact_info = get_info(soup)
    keywords = get_keywords(soup)
    
    logo_url = soup.find('img', class_='company-img').get('src')

    job_data = {
        'Job Title': job_title,
        'Location': location, #City = location[-1]
        'Salary': job_salary,
        'Experience': job_exp,
        'Company': org_name,
        'Company Link': org_link,
        'Description': job_desc,
        'Benefits': job_benefits,
        'Required Skills': job_skills,
        'Job Summary': job_summary,
        'Contact': contact_info,
        'Keywords': keywords,
        'Post date': job_post_date,
        'Logo URL': logo_url,
        'Field': None,
        
    }

    jobs_data.append(job_data)
    print(f'Finished {file_path}')


## Main

## Get new links, new files, ...

In [48]:
jobs_data = []
errors = []

In [20]:
website_pages_file = latina_pages
output_folder = r'E:\COLLEGE\PROJECT\Thay Nhan\INTERN\Crawling\data'

jobs_data = []

batch_size = 100
start = 0

with open(website_pages_file, 'r') as f:
    lines = f.readlines()[:200]
    overall_sum = len(lines)
    overall_count = 1
    for line in lines:
        page_url = line.strip()
        numbered_page_path = saving_numbered_pages(page_url, output_folder)
        
        i = 0
        overall_progress = round((overall_count / overall_sum)*100, 2)
        print(f'Overall progress: {overall_progress}%')
        while True:
            
            with open(numbered_page_path, 'r', encoding='utf-8') as file:
                html_content = file.read()
                
#             job_links = soup.find_all('a', class_='job-link')
#             links = []
#             for job in job_links:
#                 job_url = 'https://www.careerlink.vn/' + job.get('href')
#                 links += [job_url]    

            links = re.findall(r'<a\s+class="job-link clickable-outside"\s+href="([^"]+)"\s+title="[^"]*">', html_content)
            if i < len(links):
                link = links[i]
                hyperlink = "https://www.careerlink.vn" + link
                
                if extract_and_save_ids(hyperlink) == True:
                    file_path = saving_detail_file(hyperlink, output_folder)
                    
                    try:
                        # save_to_df(file_path)
                        print('Done')
                        if len(jobs_data) % batch_size == 0:
                            batch_number = len(jobs_data) // batch_size
                            batch_df = pd.DataFrame(jobs_data[start : start+batch_size])
                            print('hhihi')
                            start += batch_size
                            
                    except Exception as e:
                        print(f'Error processing {file_path}: {str(e)}')
                        errors.append(file_path)
                        
                        output_file = 'CareerLink_Errors.txt'
                        with open(output_file, 'a') as output_file:
                            output_file.write(hyperlink + '\n')
                else:
                    print('ALR EXISTED, skipping....')        
                        
                i += 1
            else:
                break
            
                
        overall_count += 1
        clear_output(wait=True)


# Save any remaining records to a final CSV file
if len(jobs_data) % batch_size != 0:
    saved_items_amt = batch_size * batch_number
    batch_df = pd.DataFrame(jobs_data[saved_items_amt:])
    print('hihii')

Overall progress: 0.5%


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\COLLEGE\\PROJECT\\Thay Nhan\\INTERN\\Crawling\\data\\latina\\2024\\2\\23\\numbered pages\\[20240223 113821]_latina_00001_page1.txt'

## Re-iterate through all available html files as a whole (for re-running case)

In [30]:
import os

root_folder = r'E:\Crawling Result\careerlink'  # Replace with your folder path

# Function to list all subfolders recursively, excluding 'numbered pages'
def list_subfolders(root_path):
    subfolders = [x[0] for x in os.walk(root_path) if 'numbered pages' not in x[0]]
    return subfolders

def find_html_files(subfolders):
    html_files = []
    for subfolder in subfolders:
        for file in os.listdir(subfolder):
            if file.lower().endswith('.txt'):
                html_files.append(os.path.join(subfolder, file))
    return html_files

subfolders = list_subfolders(root_folder)
html_files = find_html_files(subfolders)

len(html_files)

29806

In [32]:
domain = 'careerlink'
website_pages_file = careerlink_pages
output_folder = r'E:\Crawling Result'

jobs_data = []

batch_size = 100
start = 0
overall_sum = len(html_files)
overall_count = 1

i = 0
while True:
    if i < len(html_files):
        file_path = html_files[i]
        
        overall_progress = round((overall_count / overall_sum)*100, 2)
        print(f'Overall progress: {overall_progress}%')

        try:
            save_to_df(file_path)
        except Exception as e:
            print(f'Error processing {file_path}: {str(e)}')
            errors.append(file_path)

            output_file = 'CareerLink_Errors.txt'
            with open(output_file, 'a') as output_file:
                output_file.write(hyperlink + '\n')

        i += 1
        
        clear_output(wait=True)
    else:
        break

    if len(jobs_data) % batch_size == 0:
        batch_number = len(jobs_data) // batch_size
        batch_df = pd.DataFrame(jobs_data[start : start+batch_size])
        save_csv_file(batch_df, batch_number, domain, output_folder)
        start += batch_size

    overall_count += 1
    clear_output(wait=True)


# Save any remaining records to a final CSV file
if len(jobs_data) % batch_size != 0:
    saved_items_amt = batch_size * batch_number
    batch_df = pd.DataFrame(jobs_data[saved_items_amt:])
    save_csv_file(batch_df, batch_number + 1, domain, output_folder)

In [ ]:
df = pd.DataFrame(jobs_data)
print(df.shape)

# VietNam Work

## Detail View Extraction

In [1]:
def get_keywords(soup):
    job_keyword_elements = soup.find('div', class_='job-tags')
    all_tag_chips = job_keyword_elements.find_all('a')
    keywords = []

    for tag_chip in all_tag_chips:
        tag_name = tag_chip.find('span', class_='tag-name').text.strip()
        chip_list = [tag_name, tag_chip.get('href')]
        keywords.append(chip_list)
        
    return keywords

def get_summary(soup):
    job_summary_elements = soup.find('div', class_='box-summary')
    sum_boxes = job_summary_elements.find_all('div', class_='summary-item')

    test_dict = {}
    for box in sum_boxes:
        key = box.find('span', class_='content-label')
        if key != None:
            key = key.text.strip()
            if key != 'Ngành Nghề':
                val = box.find('span', class_='content')
                val = val.text.strip()
                test_dict[key]= val
            else:
                val_list = []
                vals = box.find_all('a')
                for val in vals:
                    val_list.append([val.text.strip(), val.get('href')])
                test_dict[key]= val_list
                    

    return test_dict

   
def save_to_df(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html5lib")


    # JOB TITLE
    job_title = soup.find('h1', class_='job-title').contents[0].strip()
    
    # LOCATION
    location = soup.find('div', class_='location-name').text.strip()
    location = location.split(':')[1].strip()
    
    # SALARY
    job_salary = soup.find('span', class_='salary').text.strip()
    
    # EXPERIENCE
    job_exp = None
    
    # COMPANY
    org_name = soup.find('span', class_='name').text.strip()
    
    # COMPANY LINK
    org_link = soup.find('div', class_='new-meta-company')
    org_link = org_link.find('a').get('href')
    
    # DESCRIPTION
    job_desc = soup.find('div', class_='description').text.strip()
    
    # BENEFITS
    benefit_list = soup.find_all('div', class_='benefit-name')
    job_benefits = []
    for benefit in benefit_list:
        job_benefits += [benefit.text.strip()]
        
    # REQUIRED SKILLS
    job_skills = soup.find('div', class_='requirements').text.strip()
    
    # SUMMARY
    job_summary = get_summary(soup)
    
    # CONTACT
    contact_info = {'Contact': None, 'Location': None, 'Notes': None}
    loc_div = soup.find('div', class_='job-locations')
    contact_info['Location'] = loc_div.find('div', class_='location-name').text.strip()
    
    # KEYWORDS
    keywords = get_keywords(soup)
    
    # POST DATE
    job_post_date = job_summary['Ngày Đăng Tuyển']
    
    # LOGO URL
    logo_url = soup.find('img', class_='logo').get('src')
    
    # FIELD
    job_field = job_summary['Lĩnh vực'] 
    
    job_data = {
        'Job Title': job_title,
        'Location': location, #City = location[-1]
        'Salary': job_salary,
        'Experience': job_exp,
        'Company': org_name,
        'Company Link': org_link,
        'Description': job_desc,
        'Benefits': job_benefits,
        'Required Skills': job_skills,
        'Job Summary': job_summary,
        'Contact': contact_info,
        'Keywords': keywords,
        'Post date': job_post_date,
        'Logo URL': logo_url,
        'Field': job_field,
        
    }

    jobs_data.append(job_data)
    print(f'Finished {file_path}')  

# Get links on job list

In [2]:

import pandas as pd

def scroll_to_end_and_back(driver):
    # Maximize the window
#     driver.maximize_window()
    
    # Zoom out to 25% (optional)
    driver.execute_script("document.body.style.zoom = '25%'")
    for _ in range(5):
        # Scroll down by 200px
        driver.execute_script("window.scrollBy(0, 50);")

        # Wait for a short time (adjust as needed)
        time.sleep(3)
    
    # Scroll down to the end of the page
    while True:
        # Get the current page height
        current_height = driver.execute_script("return document.body.scrollHeight")
        
        # Scroll down to the bottom of the page
        driver.execute_script(f"window.scrollTo(0, {current_height});")
        
        # Wait for a short time (adjust as needed)
        time.sleep(3)
        
        # Get the new page height after scrolling
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # If the new height is the same as the old height, you've reached the end
        if new_height == current_height:
            break
    
    # Scroll back to the start of the page
    driver.execute_script("window.scrollTo(0, 0);")

def vnwork_get_job_links(vnwork_driver):
    job_links = []
    # Find all card elements on the page
    card_elements = vnwork_driver.find_elements(By.CLASS_NAME, 'search_list')

    for card_element in card_elements:
        # Link to the job view
        try:
            link_element = card_element.find_element(By.CSS_SELECTOR, 'a')
            link = link_element.get_attribute('href')
            
            if extract_and_save_ids(link) == True:
                job_links.append(link)
            else:
                print('ALR EXISTED. Skipping....')
                
        except NoSuchElementException:
            link = None
    return job_links

## Main

In [6]:
website_pages_file = vnwork_pages
output_folder = r'E:\Crawling Result'

In [7]:
jobs_data = []
errors = []
all_links = []

In [89]:
# Open the file in append mode
with open(website_pages_file, 'r') as f:
    page_links = f.readlines()[:100]
    
vnwork_driver = webdriver.Chrome()
page_count = 1

output_file = 'VNWork_WantedLinks_Backup.txt'
with open(output_file, 'a') as output_file:
    for line in page_links:
        page_url = line.strip()
        page_progress = round((page_count * 100 / len(page_links)), 2)
        print(f'Overall progress: {page_progress}%')

        vnwork_driver.get(page_url)
        numbered_page_path = saving_numbered_pages(page_url, output_folder)
        scroll_to_end_and_back(vnwork_driver)

        new_links = vnwork_get_job_links(vnwork_driver)
        
        backup_links = new_links
        all_links += new_links

        # Write the links to the output file
        for link in new_links:
            output_file.write(link + '\n')

        clear_output(wait=True)
        page_count += 1

        
with open("VNWork_WantedLinks.txt", "w") as link_file:
    for link in all_links:
        link_file.write(link + "\n")

Overall progress: 100.0%
ALR EXISTED. Skipping....


In [14]:
jobs_data = []
batch_size = 100
start = 0

with open("VNWork_WantedLinks.txt", 'r') as f:
    lines = f.readlines()[700:]
    
overall_sum = len(lines)
overall_count = 1
batch_number = 0
    
for hyperlink in lines:
    overall_progress = round((overall_count / overall_sum)*100, 2)
    print(f'Overall progress: {overall_progress}%')
    
    file_path = saving_detail_file(hyperlink, output_folder)
    try:
        save_to_df(file_path)
        if len(jobs_data) % batch_size == 0:
            batch_number = len(jobs_data) // batch_size
            batch_df = pd.DataFrame(jobs_data[start : start+batch_size])
            save_csv_file(batch_df, batch_number, 'vietnamworks', output_folder)
            start += batch_size
    except Exception as e:
        print(f'Error processing {file_path}: {str(e)}')
        errors.append(file_path)

        
    overall_count += 1
    clear_output(wait=True)

# Save any remaining records to a final CSV file
if len(jobs_data) % batch_size != 0:
    saved_items_amt = batch_size * batch_number
    batch_df = pd.DataFrame(jobs_data[saved_items_amt:])
    save_csv_file(batch_df, batch_number + 1, 'vietnamworks', output_folder)

Overall progress: 100.0%
Finished E:\Crawling Result\vietnamworks\2023\11\3\2000\[20231103 080240]_00001_truong-phong-quan-ly-chat-luong-may-mac.txt


In [ ]:
df = pd.DataFrame(jobs_data)
df.shape

## Re-iterate through all available html files as a whole (for re-running case)

In [35]:
import os

root_folder = r'E:\Crawling Result\vietnamworks'  # Replace with your folder path

# Function to list all subfolders recursively, excluding 'numbered pages'
def list_subfolders(root_path):
    subfolders = [x[0] for x in os.walk(root_path) if 'numbered pages' not in x[0]]
    return subfolders

def find_html_files(subfolders):
    html_files = []
    for subfolder in subfolders:
        for file in os.listdir(subfolder):
            if file.lower().endswith('.txt'):
                html_files.append(os.path.join(subfolder, file))
    return html_files

subfolders = list_subfolders(root_folder)
html_files = find_html_files(subfolders)

len(html_files)

13207

In [49]:
domain = 'vietnamworks'
website_pages_file = careerlink_pages
output_folder = r'E:\Crawling Result'

jobs_data = []

batch_size = 100
start = 0
overall_sum = len(html_files)
overall_count = 1

i = 400
while True:
    if i < len(html_files):
        file_path = html_files[i]
        
        overall_progress = round((overall_count / overall_sum)*100, 2)
        print(f'Overall progress: {overall_progress}%')

        try:
            save_to_df(file_path)
        except Exception as e:
            print(f'Error processing {file_path}: {str(e)}')
            errors.append(file_path)

            output_file = 'vietnamworks_Errors.txt'
            with open(output_file, 'a') as output_file:
                output_file.write(file_path + '\n')

        i += 1
        
        clear_output(wait=True)
    else:
        break

    if len(jobs_data) % batch_size == 0:
        batch_number = len(jobs_data) // batch_size
        batch_df = pd.DataFrame(jobs_data[start : start+batch_size])
        save_csv_file(batch_df, batch_number, domain, output_folder)
        start += batch_size

    overall_count += 1
    clear_output(wait=True)


# Save any remaining records to a final CSV file
if len(jobs_data) % batch_size != 0:
    saved_items_amt = batch_size * batch_number
    batch_df = pd.DataFrame(jobs_data[saved_items_amt:])
    save_csv_file(batch_df, batch_number + 1, domain, output_folder)

In [38]:
df = pd.DataFrame(jobs_data)
df.shape

(13106, 15)


In [41]:
output_folder = r'E:\Crawling Result\2023\10\19\vietnamworks\result'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
output_file = os.path.join(output_folder, 'merged_csv.csv')
df.to_csv(output_file, index=False)

In [43]:
vnwork_df = pd.read_csv(output_file)
vnwork_df.shape

(13106, 15)

# Hotel Job

## Detail view extraction

In [13]:
def get_overview(soup):
    job_overview = soup.find('div', class_='overview-cotain')
    overview_boxes = job_overview.find_all('div', class_='media-body')

    overview_dict = {}

    for box in overview_boxes:
        key = box.find('strong').text.strip()
        if key not in ('Loại hình', 'Ngành nghề', 'Cập nhật', 'Vị trí'):
            val = box.find('br').next_sibling.strip()
        elif key == 'Cập nhật':
            val = box.find('span').text.strip()

        elif key == 'Loại hình':
            a_list = box.find_all('a')
            val = []
            for a in a_list:
                val_list = [a.text.strip(), 'https://www.hoteljob.vn' + a.get('href')]
                val += [val_list]
        else:
            val_list = [box.find('a').text.strip(), 'https://www.hoteljob.vn' + box.find('a').get('href')]
            val = val_list  # Assign the list to val
            if key == 'Vị trí':
                key = 'Cấp Bậc'
        overview_dict[key] = val

    return overview_dict, overview_dict['Nơi làm việc'], overview_dict['Ngành nghề'], overview_dict['Cập nhật']

def save_to_df(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html5lib")

    # TITLE
    job_title = soup.find('h1').text.strip()

    # DESCRIPTION, BENEFITS, SKILLS, CONTACT_INFO    
    content_blocks = soup.find_all('div', class_='content-block')
    content_block_list = []
    for block in content_blocks:
        content_block_list += [block.text.strip()]

    job_desc = content_block_list[0]
    job_benefits = content_block_list[1]
    job_skills = content_block_list[2]
    contact_info = content_block_list[3]

    # LOCATION, FIELD, POST DAY
    job_summary, location, job_field, job_post_date = get_overview(soup)


    # LOGO URL
    logo_url = soup.find('a', class_='avatar-item')
    logo_url = 'https://www.hoteljob.vn' + logo_url.find('img').get('src')

    # COMPANY NAME
    org_box = soup.find('div', class_='ntd-right-info-item')
    org_name = org_box.find('h2').text.strip()

    # COMPANY LINK
    org_link = 'https://www.hoteljob.vn' + org_box.find('a').get('href')
    org_link

    # SALARY
    ul = soup.find('ul', class_="ul-hethan-mucluong")
    job_salary = ul.find_all('li')[1].find('strong').text.strip()

    # XP, KW
    job_exp, keywords = None, None
    
    job_data = {
        'Job Title': job_title,
        'Location': location, #City = location[-1]
        'Salary': job_salary,
        'Experience': job_exp,
        'Company': org_name,
        'Company Link': org_link,
        'Description': job_desc,
        'Benefits': job_benefits,
        'Required Skills': job_skills,
        'Job Summary': job_summary,
        'Contact': contact_info,
        'Keywords': keywords,
        'Post date': job_post_date,
        'Logo URL': logo_url,
        'Field': job_field,
        
    }

    jobs_data.append(job_data)
    print(f'Finished {file_path}')  

In [14]:
errors = []
jobs_data = []
errors = []
website_pages_file = hotel_pages
output_folder = r'E:\Crawling Result'
# output_folder = r'E:\Test'

In [80]:
jobs_data = []
batch_size = 100
start = 0
with open(website_pages_file, 'r') as f:
    lines = f.readlines()[:]
    overall_sum = len(lines)
    overall_count = 1
    for line in lines:
        page_url = line.strip()
        numbered_page_path = saving_numbered_pages(page_url, output_folder)
        
        i = 0
        overall_progress = round((overall_count / overall_sum)*100, 2)
        print(f'Overall progress: {overall_progress}%')
        while True:
            
            with open(numbered_page_path, 'r', encoding='utf-8') as file:
                html_content = file.read()
            soup = BeautifulSoup(html_content, "html5lib")
                
            jobs = soup.find_all('a', class_='text-red')
            links = []
            
            for job in jobs:
                job_url = 'https://www.hoteljob.vn' + job.get('href')
                links += [job_url]
                
            if i < len(links):
                hyperlink = links[i]
                
                if extract_and_save_ids(hyperlink) == True:
                    file_path = saving_detail_file(hyperlink, output_folder)
                    
                    try:
                        save_to_df(file_path)
                        
                        if len(jobs_data) % batch_size == 0:
                            batch_number = len(jobs_data) // batch_size
                            batch_df = pd.DataFrame(jobs_data[start : start+batch_size])
                            save_csv_file(batch_df, batch_number, 'hoteljob',output_folder)
                            start += batch_size      
                                              
                    except Exception as e:
                        print(f'Error processing {file_path}: {str(e)}')
                        errors.append(file_path)
                        
                        output_file = 'HotelJob_Errors.txt'
                        with open(output_file, 'a') as output_file:
                            output_file.write(hyperlink + '\n')
                else:
                    print('ALR EXISTED, skipping....')   
                
                i += 1
            else:
                break
                
                
        overall_count += 1
        clear_output(wait=True)

# Save any remaining records to a final CSV file
if len(jobs_data) % batch_size != 0:
    saved_items_amt = batch_size * batch_number
    batch_df = pd.DataFrame(jobs_data[saved_items_amt:])
    save_csv_file(batch_df, batch_number + 1, 'hoteljob', output_folder)

Overall progress: 100.0%


In [ ]:
len(errors)

In [ ]:
df = pd.DataFrame(jobs_data)
df.head()

## Re-iterate through all available html files as a whole (for re-running case)

In [15]:
import os

root_folder = r'E:\Crawling Result\hoteljob'  # Replace with your folder path

# Function to list all subfolders recursively, excluding 'numbered pages'
def list_subfolders(root_path):
    subfolders = [x[0] for x in os.walk(root_path) if 'numbered pages' not in x[0]]
    return subfolders

def find_html_files(subfolders):
    html_files = []
    for subfolder in subfolders:
        for file in os.listdir(subfolder):
            if file.lower().endswith('.txt'):
                html_files.append(os.path.join(subfolder, file))
    return html_files

subfolders = list_subfolders(root_folder)
html_files = find_html_files(subfolders)

len(html_files)

2408

In [18]:
domain = 'hoteljob'
website_pages_file = careerlink_pages
output_folder = r'E:\Crawling Result'
errors = []

jobs_data = []

batch_size = 100
start = 0
overall_sum = len(html_files)
overall_count = 1

i = 0
while True:
    if i < len(html_files):
        file_path = html_files[i]
        
        overall_progress = round((overall_count / overall_sum)*100, 2)
        print(f'Overall progress: {overall_progress}%')

        try:
            save_to_df(file_path)
            if len(jobs_data) % batch_size == 0:
                batch_number = len(jobs_data) // batch_size
                batch_df = pd.DataFrame(jobs_data[start : start+batch_size])
                save_csv_file(batch_df, batch_number, domain, output_folder)
                start += batch_size    
                
        except Exception as e:
            print(f'Error processing {file_path}: {str(e)}')
            errors.append(file_path)

            output_file = 'HotelJob_Errors.txt'
            with open(output_file, 'a') as output_file:
                output_file.write(file_path + '\n')

        i += 1
        
        clear_output(wait=True)
    else:
        break

    overall_count += 1
    clear_output(wait=True)


# Save any remaining records to a final CSV file
if len(jobs_data) % batch_size != 0:
    saved_items_amt = batch_size * batch_number
    batch_df = pd.DataFrame(jobs_data[saved_items_amt:])
    save_csv_file(batch_df, batch_number + 1, domain, output_folder)

Overall progress: 100.0%
Error processing E:\Crawling Result\hoteljob\2023\9\30\1000\[20230930 103516]_00001_122402-duty-manager.txt: list index out of range


In [ ]:
df = pd.DataFrame(jobs_data)
print(df.shape)